# 참고자료

- https://joshdurbin.net/posts/2020-5-redis-graph-product-recommendation-opencypher/
- https://go.neo4j.com/rs/710-RRC-335/images/Neo4j_WP_Recommendations_EN_BUS.pdf?_gl=1*qnfpgj*_ga*MTY1MTQ4Mjk4NS4xNjE1MTY2MTQ1*_ga_DL38Q8KGQC*MTYxOTA0NjE5NS40LjEuMTYxOTA0NjM3Ni4w&_ga=2.171976224.242446795.1619046196-1651482985.1615166145

# 모듈 및 데이터 불러오기

In [1]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import ast
import copy
import numpy as np

import redis
from redisgraph import Node, Edge, Graph, Path

%matplotlib inline

In [2]:
purchase_data = pd.read_excel('./data/random_user_behavior_1week_gmmpg_js.xlsx')
purchase_data.head(2)

,eventtime,userid,behavior,prod_sku
0,2021-10-01 00:00:12.648,3692,view,[134]
1,2021-10-01 00:00:13.617,2921,addtocart,[94]


In [3]:
purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   eventtime  60000 non-null  datetime64[ns]
 1   userid     60000 non-null  int64         
 2   behavior   60000 non-null  object        
 3   prod_sku   60000 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.8+ MB


## 방문 고객 EDA

In [4]:
purchase_data.userid.value_counts()

4279    18
2521    18
6372    18
3159    18
3536    17
        ..
6682     1
3647     1
3792     1
1024     1
1338     1
Name: userid, Length: 7998, dtype: int64

## 구매 행동 EDA

In [5]:
purchase_data.behavior.value_counts()

# 총 5가지의 구매 행위
# 상세 페이지(detail), 결제창(checkout), 상품 추가(add), 
# 구매(purchase), 환불(refund)

view         49737
checkout      4531
addtocart     3168
transact      2556
refund           8
Name: behavior, dtype: int64

## 상품 종류 EDA

In [6]:
purchase_data.prod_sku.value_counts()

# check_out과 puchase에서 상품 종류 복수개 list 존재
# graph-data import 진행하는 경우, Data 분리 필요
# 이후 같은 구매라고 식별하기 위해서 behavior time과 고객 ID를 key값으로 하여 식별 가능함 

[159]                              316
[145]                              316
[63]                               305
[57]                               305
[68]                               301
                                  ... 
[2, 139, 116]                        1
[66, 45, 30, 105, 83]                1
[82, 121, 78, 60, 102, 61, 10]       1
[90, 169, 60, 110, 65, 23, 146]      1
[51, 185]                            1
Name: prod_sku, Length: 6521, dtype: int64

In [7]:
purchase_data.loc[purchase_data.behavior == 'detail', 'prod_sku'].apply(lambda x: len(ast.literal_eval(x))).sort_values(ascending=False)

Series([], Name: prod_sku, dtype: object)

In [8]:
purchase_data.loc[purchase_data.behavior == 'checkout', 'prod_sku'].apply(lambda x: len(ast.literal_eval(x))).sort_values(ascending=False)

59999    10
35696    10
37120    10
37101    10
37075    10
         ..
55869     1
7164      1
44010     1
44039     1
54792     1
Name: prod_sku, Length: 4531, dtype: int64

In [9]:
purchase_data.loc[purchase_data.behavior == 'purchase', 'prod_sku'].apply(lambda x: len(ast.literal_eval(x))).sort_values(ascending=False)

Series([], Name: prod_sku, dtype: object)

In [10]:
purchase_data.loc[purchase_data.behavior == 'add', 'prod_sku'].apply(lambda x: len(ast.literal_eval(x))).sort_values(ascending=False)

Series([], Name: prod_sku, dtype: object)

In [11]:
purchase_data.loc[purchase_data.behavior == 'refund', 'prod_sku'].apply(lambda x: len(ast.literal_eval(x))).sort_values(ascending=False)

56523    1
53061    1
22363    1
22272    1
19025    1
9329     1
8741     1
3617     1
Name: prod_sku, dtype: int64

# Data Preprocessing

## purchase 행동 식별 ID 부여

- 구분되는 Node 및 Edge 생성을 위함
- 데이터가 실시간으로 들어오는 경우, 신규 데이터 처리 어떻게 할 것인지?
    - 추가 data에서 구매ID 필요
    - Test에서는 순서대로 0부터 ID 값을 부여하는 것으로 정함 

In [12]:
copy_data = copy.deepcopy(purchase_data)
copy_data.head(2)

,eventtime,userid,behavior,prod_sku
0,2021-10-01 00:00:12.648,3692,view,[134]
1,2021-10-01 00:00:13.617,2921,addtocart,[94]


In [13]:
copy_data.loc[copy_data.behavior == 'transact', 'purchase_id'] = range(len(copy_data[copy_data.behavior == 'transact']))

In [14]:
copy_data.purchase_id[~copy_data.purchase_id.isnull()]

15          0.0
82          1.0
147         2.0
164         3.0
171         4.0
          ...  
59921    2551.0
59943    2552.0
59947    2553.0
59984    2554.0
59995    2555.0
Name: purchase_id, Length: 2556, dtype: float64

## checkout and purchase 행동의 상품 항목 분리

In [15]:
copy_data["product_num"] = copy_data.prod_sku.apply(lambda x: len(ast.literal_eval(x)))

In [16]:
copy_data.loc[copy_data.product_num >= 2, 'prod_sku'].apply(lambda x: ast.literal_eval(x))

15                      [117, 109, 139, 108, 77, 1, 49]
26       [157, 127, 4, 102, 62, 101, 43, 163, 181, 147]
37          [55, 31, 56, 13, 36, 125, 18, 101, 155, 45]
62                                    [44, 114, 94, 98]
65                                        [167, 47, 24]
                              ...                      
59982                      [77, 32, 34, 70, 85, 7, 194]
59984                                 [22, 45, 98, 168]
59994                        [73, 180, 146, 109, 99, 9]
59995                     [10, 8, 9, 61, 53, 1, 14, 78]
59999      [117, 46, 69, 116, 85, 82, 56, 148, 139, 99]
Name: prod_sku, Length: 6329, dtype: object

In [17]:
# 분리 대상 index 
div_idx = copy_data.loc[copy_data.product_num >= 2, 'prod_sku'].index

In [18]:
chg_li = []

for i in div_idx:
    for c in range(copy_data.loc[i, 'product_num']):
        temp_dict = copy_data.loc[i].to_dict()
        temp_sku = ast.literal_eval(copy_data.loc[i, 'prod_sku'])[c] # 복수개 상품 중 한개
        
        temp_dict['prod_sku'] = '['+str(temp_sku)+']' # 해당 튜플에 상품 한개씩 담에 여러개의 튜플 생성 목적
        
        chg_li.append(temp_dict)

In [19]:
copy_data.drop(div_idx, inplace=True) # 분리 대상 튜플 삭제 후, 분리 데이터 chg_li와 결합 준비 
copy_data.index = range(len(copy_data))

In [20]:
chg_df = pd.DataFrame(chg_li)
chg_df.head(2)

,eventtime,userid,behavior,prod_sku,purchase_id,product_num
0,2021-10-01 00:03:00.661,5655,transact,[117],0.0,7
1,2021-10-01 00:03:00.661,5655,transact,[109],0.0,7


In [21]:
input_data = pd.concat([copy_data, chg_df])
input_data.index=range(len(input_data))

print('변경전 데이터 수:', len(copy_data))
print('변경후 데이터 수:', len(input_data))
input_data.head(2)

변경전 데이터 수: 53671
변경후 데이터 수: 90653


,eventtime,userid,behavior,prod_sku,purchase_id,product_num
0,2021-10-01 00:00:12.648,3692,view,[134],NaN,1
1,2021-10-01 00:00:13.617,2921,addtocart,[94],NaN,1


# Input Data 
- Kinds of Node: Product, Customer, Order 
- Kinds of Edge: Refund, Detail, Checkout, Purchase, Add + Contain between Order and Product

In [22]:
# RedisGraph Instance 생성
r = redis.Redis(host='127.0.0.1', port=6379, db=1)

In [23]:
purchase_graph = Graph('product_recommendation', r)

## Generate Nodes

### Node ID 

#### Product 

In [24]:
prd_nodes = input_data.prod_sku.unique()

prd_nodes_id = []

for n, prd in enumerate(prd_nodes):
    temp_dict = {'prod_sku': prd, 'prod_sku_id': n}
    prd_nodes_id.append(temp_dict)
    
print(prd_nodes_id[:10])

[{'prod_sku': '[134]', 'prod_sku_id': 0}, {'prod_sku': '[94]', 'prod_sku_id': 1}, {'prod_sku': '[178]', 'prod_sku_id': 2}, {'prod_sku': '[16]', 'prod_sku_id': 3}, {'prod_sku': '[159]', 'prod_sku_id': 4}, {'prod_sku': '[169]', 'prod_sku_id': 5}, {'prod_sku': '[59]', 'prod_sku_id': 6}, {'prod_sku': '[65]', 'prod_sku_id': 7}, {'prod_sku': '[36]', 'prod_sku_id': 8}, {'prod_sku': '[26]', 'prod_sku_id': 9}]


In [25]:
prd_nodes_id_df = pd.DataFrame(prd_nodes_id)
prd_nodes_id_df.head(2)

,prod_sku,prod_sku_id
0,[134],0
1,[94],1


In [26]:
prd_nodes_id_df.prod_sku.value_counts()

[81]     1
[88]     1
[187]    1
[89]     1
[155]    1
        ..
[13]     1
[7]      1
[174]    1
[196]    1
[125]    1
Name: prod_sku, Length: 200, dtype: int64

In [27]:
input_data = pd.merge(input_data, prd_nodes_id_df, how='left', on='prod_sku')
input_data.head(2)

,eventtime,userid,behavior,prod_sku,purchase_id,product_num,prod_sku_id
0,2021-10-01 00:00:12.648,3692,view,[134],NaN,1,0
1,2021-10-01 00:00:13.617,2921,addtocart,[94],NaN,1,1


#### Customer

In [28]:
cust_nodes = input_data.userid.unique()

cust_nodes_id = []

for n, cust in enumerate(cust_nodes):
    temp_dict = {'userid': cust, 'cust_id': n}
    cust_nodes_id.append(temp_dict)
    
print(cust_nodes_id[:10])

[{'userid': 3692, 'cust_id': 0}, {'userid': 2921, 'cust_id': 1}, {'userid': 4185, 'cust_id': 2}, {'userid': 7188, 'cust_id': 3}, {'userid': 5847, 'cust_id': 4}, {'userid': 1296, 'cust_id': 5}, {'userid': 3511, 'cust_id': 6}, {'userid': 6979, 'cust_id': 7}, {'userid': 2200, 'cust_id': 8}, {'userid': 2211, 'cust_id': 9}]


In [29]:
cust_nodes_id_df = pd.DataFrame(cust_nodes_id)
cust_nodes_id_df.head(2)

,userid,cust_id
0,3692,0
1,2921,1


In [30]:
input_data = pd.merge(input_data, cust_nodes_id_df, how='left', on='userid')
input_data.head(2)

,eventtime,userid,behavior,prod_sku,purchase_id,product_num,prod_sku_id,cust_id
0,2021-10-01 00:00:12.648,3692,view,[134],NaN,1,0,0
1,2021-10-01 00:00:13.617,2921,addtocart,[94],NaN,1,1,1


#### Order
- purchase_id 활용 

### Input Nodes Data 

In [31]:
for prd in prd_nodes_id:
    
    prd_id = prd['prod_sku_id']
    prd_name =  prd['prod_sku']

    query = """CREATE (:product {condition})""".format(condition = '{'+f'id: {prd_id}, name: \'{prd_name}\''+'}')
    
    r.execute_command('GRAPH.QUERY', 'product_recommendation', query)

In [32]:
print('number of product node: ', len(prd_nodes_id))

number of product node:  200


In [33]:
query = """MATCH (p:product)
           RETURN size(collect(p.name))"""

result = purchase_graph.query(query)

In [34]:
result.pretty_print()

+--------------------------+
| b'size(collect(p.name))' |
+--------------------------+
|           200            |
+--------------------------+

Cached execution 0.0
internal execution time 2.140333


##### Customer 

In [35]:
for cust in cust_nodes_id:
    
    customer_id = cust['cust_id']
    cust_name =  cust['userid']

    query = """CREATE (:customer {condition})""".format(condition = '{'+f'id: {customer_id}, name: \'{cust_name}\''+'}')
    
    r.execute_command('GRAPH.QUERY', 'product_recommendation', query)

In [36]:
print('number of customer node: ', len(cust_nodes_id))

number of customer node:  7998


In [37]:
query = """MATCH (p:customer)
           RETURN size(collect(p.name))"""

result = purchase_graph.query(query)

In [38]:
result.pretty_print()

+--------------------------+
| b'size(collect(p.name))' |
+--------------------------+
|           7998           |
+--------------------------+

Cached execution 0.0
internal execution time 16.551584


##### Order 

In [39]:
for ord_n in input_data.purchase_id.unique():
    try:
        ord_n_id = int(ord_n)
        e_time = input_data.loc[input_data.purchase_id == ord_n_id, 'eventtime'].item()
    
        ord_n_name = '['+str(ord_n)+']'
        
        query = """CREATE (:order {condition})""".format(condition = '{'+f'id: {ord_n_id}, name: \'{ord_n_name}\', time: \'{e_time}\''+'}')
#         print(query)
        r.execute_command('GRAPH.QUERY', 'product_recommendation', query)
    
    except ValueError:
        pass

In [40]:
print('number of order node: ', len(input_data.loc[~input_data.purchase_id.isnull(), 'purchase_id'].unique()))

number of order node:  2556


In [41]:
query = """MATCH (p:order)
           RETURN size(collect(p.name))"""

result = purchase_graph.query(query)

In [42]:
result.pretty_print()

+--------------------------+
| b'size(collect(p.name))' |
+--------------------------+
|           277            |
+--------------------------+

Cached execution 0.0
internal execution time 6.476292


## Generate Edges
- 고객 개개인을 기준으로 구매 행동 Edge 생성

In [43]:
input_data.head()

,eventtime,userid,behavior,prod_sku,purchase_id,product_num,prod_sku_id,cust_id
0,2021-10-01 00:00:12.648,3692,view,[134],NaN,1,0,0
1,2021-10-01 00:00:13.617,2921,addtocart,[94],NaN,1,1,1
2,2021-10-01 00:00:29.639,4185,view,[178],NaN,1,2,2
3,2021-10-01 00:00:41.871,7188,view,[16],NaN,1,3,3
4,2021-10-01 00:00:42.624,5847,addtocart,[159],NaN,1,4,4


### 사용자 행동에 대한 Edges

In [44]:
input_data.behavior.value_counts()

view         49737
checkout     24931
transact     12809
addtocart     3168
refund           8
Name: behavior, dtype: int64

In [45]:
# 사용자 행동에 대한 Edges 생성 - Detail, Checkout, Puchase, Add
# datetime -> int 변경

# 1. 인기 상품 : 많이 본 상품, 많이 산 상품 
# 2. 관련 상품 : 해당 상품을 본 사람들이 본 다른 상품 / 해당 상품을 본 사람들이 산 다른 상품 / 해당 상품을 산 사람들이 본 다른 상품 / 해당 상품을 산 사람들이 산 다른 상품 
# 3. 기간에 대한 조건 설정 



for c1, c2, c3, c4, c5, c6, c7, c8 in input_data.to_records(index=False):
    if c3 != 'transact':
        query = """MATCH (c:customer {customer_id})
                   MATCH (prd:product {prod_id})
                   MERGE (c)-[:{behavior} {t}]->(prd)""".format(customer_id = '{'+f'id: {c8}'+'}', prod_id = '{'+f'id: {c7}'+'}', behavior = c3, t='{'+f'time: \'{c1}\''+'}')
    #     print(query)

        r.execute_command('GRAPH.QUERY', 'product_recommendation', query)
    else:
        try:
            query = """MATCH (c:customer {customer_id})
                       MATCH (ord:order {ord_id})
                       MERGE (c)-[:{behavior} {t}]->(ord)""".format(customer_id = '{'+f'id: {c8}'+'}', ord_id = '{'+f'id: {int(c5)}'+'}', behavior = c3, t='{'+f'time: \'{c1}\''+'}')
        #     print(query)

            r.execute_command('GRAPH.QUERY', 'product_recommendation', query)    
        except ValueError:
            pass
    

### Order node와 상품 간의 관계에 대한 Edges

In [46]:
# Order node와 상품 간의 관계 

for c1, c2, c3, c4, c5, c6, c7, c8 in input_data.to_records(index=False):
    try:
        query = """MATCH (o:order {purchase_id})
                   MATCH (prd:product {prod_id})
                   MERGE (o)-[:contain {t}]->(prd)""".format(purchase_id = '{'+f'id: {int(c5)}'+'}', prod_id = '{'+f'id: {c7}'+'}', t='{'+f'time: \'{c1}\''+'}')
#         print(query)
        r.execute_command('GRAPH.QUERY', 'product_recommendation', query)
    except ValueError:
        pass